In [1]:
import numpy as np
import pandas as pd
import scipy.io

In [2]:
# downloaded from https://data.mendeley.com/datasets/p7ft2bvphx/1
a = scipy.io.loadmat('data/empirical/SPKcounts_V20160929cue_MW_250X250ms.mat')
b = scipy.io.loadmat('data/empirical/SPKcounts_V20160930cue_MW_250X250ms.mat')
c = scipy.io.loadmat('data/empirical/SPKcounts_V20161005cue_MW_250X250ms.mat')
d = scipy.io.loadmat('data/empirical/SPKcounts_V20161017cue_MW_250X250ms.mat')
e = scipy.io.loadmat('data/empirical/SPKcounts_W20160112cue_MW_250X250ms.mat')
f = scipy.io.loadmat('data/empirical/SPKcounts_W20160113cue_MW_250X250ms.mat')
g = scipy.io.loadmat('data/empirical/SPKcounts_W20160121cue_MW_250X250ms.mat')
h = scipy.io.loadmat('data/empirical/SPKcounts_W20160122cue_MW_250X250ms.mat')

In [9]:
dfs = []
columns = ['sid', 'block', 'trial', 'left', 'right', 'correct', 'reversal_at_trial', 'monkey_accuracy']

sid = 0
for file in [a,b,c,d,e,f,g,h]:
    df = pd.DataFrame(columns=columns)
    data = file['Y']
    bid = data[:,7]
    block = data[:,9]
    trial = data[:,5]
    letter = data[:,0]
    location = data[:,1]
    best_chosen = data[:,4]
    block_completed = data[:,12]
    # reversal_at_trial = data[:,6]
    idxs = np.argwhere(data[:,6]==1)[:,0]
    reversal_at_trial = {}
    for idx in idxs:
        bb = bid[idx]
        tt = trial[idx]
        reversal_at_trial[bb]=tt
    # print(idxs, reversal_at_trial)
    # raise
    # reversal_at_trial = []
    # for bb in np.unique(bid):
    #     idx = np.argwhere(data[:,6]==1)[bb-1][0]
    #     reversal_at_trial.append(trial[idx])
    # print(reversal_at_trial)
    left = []
    right = []
    correct = []
    for row in range(data.shape[0]):
        if location[row]==0 and letter[row]==0:
            left.append('A')
            right.append('B')
        if location[row]==0 and letter[row]==1:
            left.append('B')
            right.append('A')
        if location[row]==1 and letter[row]==0:
            left.append('B')
            right.append('A')
        if location[row]==1 and letter[row]==1:
            left.append('A')
            right.append('B')
        # if block[row] == 2:  # where
        if best_chosen[row]==1 and location[row]==0:
            correct.append('left')
        if best_chosen[row]==1 and location[row]==1:
            correct.append('right')
        if best_chosen[row]==0 and location[row]==0:
            correct.append('right')
        if best_chosen[row]==0 and location[row]==1:
            correct.append('left')
        # if block[row] == 1:  # what
        #     if best_chosen[row]==1 and letter[row]==0:
        #         correct.append('left')
        #     if best_chosen[row]==1 and letter[row]==1:
        #         correct.append('right')
        #     if best_chosen[row]==0 and letter[row]==0:
        #         correct.append('right')
        #     if best_chosen[row]==0 and letter[row]==1:
        #         correct.append('left')
        # print(f"block {block[row]}, left {left[-1]}, right {right[-1]}, chose {best_chosen[row]}, correct {correct[-1]}") 
    df['bid'] = bid
    df['block'] = block
    df['trial'] = trial
    df['left'] = left
    df['right'] = right
    df['correct'] = correct
    df['monkey_accuracy'] = best_chosen
    df['block_completed'] = block_completed
    df = df[df['block_completed'] != 0]
    temp = []
    for bb in df['bid'].unique():
        for t in df.query("bid==@bb")['trial'].unique():
            temp.append(reversal_at_trial[bb])
        # print(bid, t)
    df['reversal_at_trial'] = temp
    df['sid'] = sid * np.ones((df.shape[0], 1), dtype=int)
    dfs.append(df)
    sid += 1

behavioral_data = pd.concat(dfs, ignore_index=True)
# behavioral_data['left'] = behavioral_data['letter'].replace('A', -1)
# behavioral_data['left'] = behavioral_data['letter'].replace('B', 1)
# behavioral_data['right'] = behavioral_data['location'].replace('A', -1)
# behavioral_data['right'] = behavioral_data['location'].replace('B', 1)
behavioral_data['block'] = behavioral_data['block'].replace(1, 'what')
behavioral_data['block'] = behavioral_data['block'].replace(2, 'where')
# behavioral_data = behavioral_data[behavioral_data['block_completed'] != 0]
behavioral_data = behavioral_data.drop(columns='block_completed')


In [14]:
behavioral_data.query("sid==1 & bid==10")[20:60]

,sid,block,trial,left,right,correct,reversal_at_trial,monkey_accuracy,bid
2420,1,what,21,A,B,left,38,1,10
2421,1,what,22,A,B,left,38,1,10
2422,1,what,23,A,B,left,38,1,10
2423,1,what,24,A,B,left,38,1,10
2424,1,what,25,A,B,left,38,1,10
2425,1,what,26,B,A,right,38,1,10
2426,1,what,27,A,B,left,38,1,10
2427,1,what,28,A,B,left,38,1,10
2428,1,what,29,A,B,left,38,1,10
2429,1,what,30,A,B,left,38,1,10


In [11]:
behavioral_data.to_pickle("data/empirical.pkl")

In [13]:
def check_opposite(row):
    return (row['left'] == 'A' and row['right'] == 'B') or \
           (row['left'] == 'B' and row['right'] == 'A')

# Apply the check to all rows
all_opposite = behavioral_data.apply(check_opposite, axis=1).all()

print("All rows contain opposite letters:", all_opposite)

All rows contain opposite letters: True
